In [ ]:
# Ensure `df` exists, normalize columns, and build augmented DataFrame
from pathlib import Path
import pandas as pd

CSV_FALLBACK = Path("/Users/leonardo.galebe/Desktop/WMC/aegis_products.csv")

if 'df' in globals() and isinstance(df, pd.DataFrame):
    base_df = df.copy()
elif 'PRODUCTS' in globals():
    base_df = pd.DataFrame(PRODUCTS)
elif CSV_FALLBACK.exists():
    base_df = pd.read_csv(CSV_FALLBACK)
else:
    raise FileNotFoundError(
        "Could not initialize DataFrame: define `PRODUCTS` or create "
        f"the CSV fallback at {CSV_FALLBACK}"
    )

COL_REMAP = {
    "intended purpose": "intended_purpose",
    "fitness of use": "fitness_of_use",
}

def _norm_cols_local(frame: pd.DataFrame) -> pd.DataFrame:
    return frame.rename(columns=COL_REMAP)

base_df = _norm_cols_local(base_df)

SECURITY_SURV_EVID = (
    "Homes without security show far higher burglary risk; offender surveys ~83% check for alarms "
    "and ~60% would avoid if an alarm is present."
)
SECURITY_SURV_POLICY = (
    "Deterrence varies by context. Mobile/AI towers can produce false alerts in busy scenes; "
    "define retention, auditing, and public-notice policies."
)
SECURITY_SURV_SRC = "PMC (home security risk/deterrence); Governance best practices"

DASHCAM_EVID = (
    "Fleet use of in-cab/dual-facing cameras ~63% in 2025 surveys. Crash video improves objectivity "
    "for insurance/legal investigations."
)
DASHCAM_POLICY = (
    "Privacy and data-governance needed (who is recorded, when; retention). For LE, ensure chain-of-custody."
)
DASHCAM_SRC = "Industry fleet surveys; Crash forensics policy guidance"

TRAFFIC_CAM_EVID = (
    "Meta-analyses: right-angle injury crashes ↓~29%; right-angle crashes ↓~24%; rear-end crashes ↑~32%; "
    "total crashes ↓~12% (site dependent)."
)
TRAFFIC_CAM_POLICY = (
    "Use engineering review, warning signage, and calibration audits to balance safety gains vs. rear-end risk."
)
TRAFFIC_CAM_SRC = "FHWA red-light camera meta-analyses"

CATEGORY_TEXT = {
    "Security Surveillance": (SECURITY_SURV_EVID, SECURITY_SURV_POLICY, SECURITY_SURV_SRC),
    "Personnel Surveillance": (SECURITY_SURV_EVID, SECURITY_SURV_POLICY, SECURITY_SURV_SRC),
    "Dashcam": (DASHCAM_EVID, DASHCAM_POLICY, DASHCAM_SRC),
    "Traffic Cameras": (TRAFFIC_CAM_EVID, TRAFFIC_CAM_POLICY, TRAFFIC_CAM_SRC),
}

def _apply_policy_bundle(row):
    evid, policy, src = CATEGORY_TEXT.get(row.get('category'), ("", "", ""))
    return pd.Series({
        'evidence_stats': evid,
        'policy_notes': policy,
        'sources_label': src,
    })

policy_cols = base_df.apply(_apply_policy_bundle, axis=1)

df = base_df.join(policy_cols)
df_aug = df.copy()

print("Initialized df and df_aug with policy annotations:")
print(df_aug[['category', 'evidence_stats', 'policy_notes', 'sources_label']].head())


In [5]:
import pandas as pd

# --- normalize original column names if needed (safe even if already normalized) ---
def _norm_cols(df):
    return df.rename(columns={
        "intended purpose": "intended_purpose",
        "fitness of use": "fitness_of_use"
    })

df = _norm_cols(df)

# ===============================
# Evidence/Policy text blocks + citations
# ===============================

SECURITY_SURV_EVID = (
    "Homes without security show far higher burglary risk; offender surveys ~83% check for alarms "
    "and ~60% would avoid if an alarm is present."
)
SECURITY_SURV_POLICY = (
    "Deterrence varies by context. Mobile/AI towers can produce false alerts in busy scenes; "
    "define retention, auditing, and public-notice policies."
)
SECURITY_SURV_EVID_SRC  = "PMC (home security risk/deterrence)"
SECURITY_SURV_POLICY_SRC = "General governance best practices"

DASHCAM_EVID = (
    "Fleet use of in-cab/dual-facing cameras ~63% in 2025 surveys. Crash video improves objectivity "
    "for insurance/legal investigations."
)
DASHCAM_POLICY = (
    "Privacy/data-governance needed (who is recorded, when; retention). For LE, ensure chain-of-custody."
)
DASHCAM_EVID_SRC  = "Industry fleet surveys + crash forensics"
DASHCAM_POLICY_SRC = "Policy best practices"

TRAFFIC_CAM_RLC_EVID = (
    "Meta-analyses: right-angle injury crashes ↓~29%; right-angle crashes ↓~24%; rear-end crashes ↑~32%; "
    "total crashes ↓~12% (site dependent)."
)
TRAFFIC_CAM_RLC_POLICY = (
    "Balance right-angle safety gains vs rear-end increases. Require calibration, signage, due process, and audits."
)
TRAFFIC_CAM_RLC_EVID_SRC  = "ScienceDirect (RLC meta-analyses)"
TRAFFIC_CAM_RLC_POLICY_SRC = "Program governance"

TRAFFIC_CAM_SPEED_SCHOOL_EVID = (
    "School-zone cameras (NYC case): speeding ↓63%; crashes ↓15%; injuries ↓17%; fatalities ↓55% at treated sites."
)
TRAFFIC_CAM_SPEED_POLICY = (
    "Program design matters (hours, signage, thresholds, equity). Expect political debate; report transparently."
)
TRAFFIC_CAM_SPEED_EVID_SRC  = "Federal Highway Administration (NYC)"
TRAFFIC_CAM_SPEED_POLICY_SRC = "FHWA guidance"

ALPR_EVID = (
    "Automates stolen/wanted hits and provides real-time and historical investigative queries."
)
ALPR_POLICY = (
    "Risks: false positives, retention/mission creep, privacy/civil liberties; mandate audit trails, access controls, "
    "short retention, and oversight."
)
ALPR_EVID_SRC  = "Operational policy briefs"
ALPR_POLICY_SRC = "Ford School STPP (governance & risks)"

PERSONNEL_BWC_EVID = (
    "Mixed rigorous evidence: DC randomized trial found no detectable average effect on force/complaints; "
    "other sites report sizable reductions in complaints/force."
)
PERSONNEL_BWC_POLICY = (
    "Outcomes depend on activation rules, supervision, disclosure. Secure uploads and chain-of-custody are essential."
)
PERSONNEL_BWC_EVID_SRC  = "bwc.thelab.dc.gov + NIJ summaries"
PERSONNEL_BWC_POLICY_SRC = "NIJ/agency policy"

DFR_EVID = (
    "Drone-as-First-Responder: typical arrivals ~100–210s reported; often first on scene; some calls resolved without "
    "dispatching a patrol unit. Adoption growing."
)
DFR_POLICY = (
    "Airspace/regulatory limits; community privacy concerns; require transparent SOPs, force boundaries, logging & audits."
)
DFR_EVID_SRC  = "DJI Enterprise Insights; OPSI; program reports"
DFR_POLICY_SRC = "Program governance"

# --- Routing logic by category + product name ---
def _blocks(row):
    cat  = str(row.get("category","")).strip().lower()
    name = str(row.get("name","")).strip().lower()

    if "security surveillance" in cat:
        return SECURITY_SURV_EVID, SECURITY_SURV_POLICY, SECURITY_SURV_EVID_SRC, SECURITY_SURV_POLICY_SRC

    if "dashcam" in cat:
        return DASHCAM_EVID, DASHCAM_POLICY, DASHCAM_EVID_SRC, DASHCAM_POLICY_SRC

    if "traffic cameras" in cat:
        if "red light" in name:
            return TRAFFIC_CAM_RLC_EVID, TRAFFIC_CAM_RLC_POLICY, TRAFFIC_CAM_RLC_EVID_SRC, TRAFFIC_CAM_RLC_POLICY_SRC
        if "school" in name:
            return TRAFFIC_CAM_SPEED_SCHOOL_EVID, TRAFFIC_CAM_SPEED_POLICY, TRAFFIC_CAM_SPEED_EVID_SRC, TRAFFIC_CAM_SPEED_POLICY_SRC
        if "speed" in name or "radar" in name:
            return TRAFFIC_CAM_SPEED_SCHOOL_EVID, TRAFFIC_CAM_SPEED_POLICY, TRAFFIC_CAM_SPEED_EVID_SRC, TRAFFIC_CAM_SPEED_POLICY_SRC
        if "alpr" in name or "license plate" in name:
            return ALPR_EVID, ALPR_POLICY, ALPR_EVID_SRC, ALPR_POLICY_SRC
        # fallback for other traffic cams
        return TRAFFIC_CAM_RLC_EVID, TRAFFIC_CAM_RLC_POLICY, TRAFFIC_CAM_RLC_EVID_SRC, TRAFFIC_CAM_RLC_POLICY_SRC

    if "personnel surveillance" in cat:
        return PERSONNEL_BWC_EVID, PERSONNEL_BWC_POLICY, PERSONNEL_BWC_EVID_SRC, PERSONNEL_BWC_POLICY_SRC

    if "drone deployment" in cat:  # Wraith
        return DFR_EVID, DFR_POLICY, DFR_EVID_SRC, DFR_POLICY_SRC

    return "", "", "", ""

# --- Create a NEW augmented DataFrame (leaves original df untouched) ---
df_aug = df.copy()
df_aug["evidence_stats"]    = ""
df_aug["policy_notes"]      = ""
df_aug["evidence_citation"] = ""
df_aug["policy_citation"]   = ""

for i, r in df_aug.iterrows():
    ev, pol, ev_src, pol_src = _blocks(r)
    df_aug.at[i, "evidence_stats"]    = ev
    df_aug.at[i, "policy_notes"]      = pol
    df_aug.at[i, "evidence_citation"] = ev_src
    df_aug.at[i, "policy_citation"]   = pol_src

print("Augmented DataFrame ready → df_aug")
display(df_aug.head(3))


NameError: name 'df' is not defined